### Dataset : Flickr 8k images

In [1]:
filename = "captions.txt"
file = open(filename, 'r')
doc = file.read()

In [2]:
data=doc.split('\n')
print(len(data))

40456


In [5]:
li_data=[]
for i in range(len(data)):
    modified_data = data[i].split(',')
    li_data.append(modified_data)

print(li_data[0])

['1000268201_693b08cb0e.jpg', 'A child in a pink dress is climbing up a set of stairs in an entry way .']


In [6]:
# Workflow to create a dictionary in which the key is the image ID and the value is a list of respective 5 captions:
# step1 : create a list which consist of only image IDs.
# step2 : create a id dictionary in which key is image ID and value is the count of captions i.e; 5
# step3 : create a list of all the unique IDs out from the id dictionary.
# step4 : Finally run two loops and create a id_caption_dict in which the required key value pair is present.

In [11]:
import collections
id_data=[] # list in which all the ids are stored
for i in range(len(li_data)):
    id_data.append(li_data[i][0])
print(len(id_data))

40456


In [12]:
id_dict={}
id_dict = collections.Counter(id_data)
id_dict_key = [] # list in which all the unique ids are stored
for key in id_dict:
    id_dict_key.append(key)
len(id_dict_key) 

8092

In [29]:
caption_list = [] # list in which all the captions are stored
for i in range(len(li_data)-1):
    caption_list.append('start_caption '+li_data[i][1].lower()+' end_caption')
print(caption_list[0])
print(len(caption_list))

start_caption a child in a pink dress is climbing up a set of stairs in an entry way . end_caption
40455


In [30]:
id_caption_dict = {} # dict in which id is stored as a key and list of all the corresponding 5 captions is stored as value
for j in range(1,8092):
    caption_li=[]
    for i in range(5*(j-1),5*j):
        caption_li.append('start_caption '+li_data[i][1].lower()+' end_caption')
    id_caption_dict[id_dict_key[j-1]]=caption_li

id_caption_dict[id_dict_key[1]]

['start_caption a black dog and a spotted dog are fighting end_caption',
 'start_caption a black dog and a tri-colored dog playing with each other on the road . end_caption',
 'start_caption a black dog and a white dog with brown spots are staring at each other in the street . end_caption',
 'start_caption two dogs of different breeds looking at each other on the road . end_caption',
 'start_caption two dogs on pavement moving toward each other . end_caption']

In [31]:
vocabulary = set()
for key in id_caption_dict.keys():
    [vocabulary.update(d.split()) for d in id_caption_dict[key]]
print('Vocabulary Size: ' , len(vocabulary))

Vocabulary Size:  8873


In [32]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in caption_list:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 8874 -> 1940


In [34]:
word_to_index = {}
index_to_word = {}

index=1
for word in vocab:
    word_to_index[word]=index
    index_to_word[index]=word
    index=index+1

print(len(word_to_index))

1940


In [35]:
vocab_size = len(index_to_word) + 1 # one for appended 0's
vocab_size

1941

In [36]:
len_of_caption_in_list=[]
for i in range(len(caption_list)):
    len_of_caption_in_list.append(len(caption_list[i].split(' ')))
max_len_of_caption = max(len_of_caption_in_list)
print(max_len_of_caption)

36


In [37]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model

model = InceptionV3(weights='imagenet')
filename = 'inception_V3_model.h5'
model.save(filename)
model.load_weights('inception_V3_model.h5')
model_inception = Model(model.input, model.layers[-2].output)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [38]:
model_inception.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [42]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [100]:
id_imgarr_dict = {}
for i in range(len(id_dict_key)):
    img = image.load_img('images/'+id_dict_key[i], target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = model_inception.predict(x)
    x = np.reshape(predictions, predictions.shape[1])
    id_imgarr_dict[id_dict_key[i]]=x

In [45]:
id_imgarr_dict[id_dict_key[0]].shape

(2048,)

In [46]:
X_imgarr = []
for key in id_imgarr_dict:
    X_imgarr.append(id_imgarr_dict[key])
X_imgarr = np.reshape(X_imgarr, (len(X_imgarr),)+(2048,))

In [47]:
X_imgarr.shape

(8091, 2048)

In [48]:
def read_glove_vec(glove_file):
    with open(glove_file, encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index_glove_vocab = {}
        index_to_words_glove_vocab = {}
        for w in sorted(words):
            words_to_index_glove_vocab[w] = i
            index_to_words_glove_vocab[i] = w
            i = i + 1
    return words_to_index_glove_vocab, index_to_words_glove_vocab, word_to_vec_map, words

In [49]:
word_to_index_glove_vocab, index_to_word_glove_vocab, word_to_vec_map, words = read_glove_vec('glove.6B.200d.txt')

In [51]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
vocab_caption = np.zeros((vocab_size, embedding_dim))

for word, i in word_to_index.items():
    #if i < max_words:
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        vocab_caption[i] = embedding_vector

In [52]:
len(vocab_caption)

1941

In [53]:
X=np.reshape(vocab_caption,(len(vocab_caption),)+(200,))
X.shape

(1941, 200)

### MODEL ARCHITECTURE

<img src='model_architecture.png'>

In [69]:
def model(vocab_size,embedding_dim, max_len_of_caption):
    
    input_1 = Input(shape=(2048,))
    X = Dropout(0.5)(input_1)
    X = Dense(256,activation='relu')(X)
    
    input_2 = Input(shape=(max_len_of_caption,))
    X1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(input_2)
    X1 = Dropout(0.5)(X1)
    X1 = LSTM(256)(X1)
    
    X_X1 = add([X,X1])
    X_X1 = Dense(256,activation='relu')(X_X1)
    out = Dense(vocab_size, activation='softmax')(X_X1)
    
    model = Model(inputs=[input_1,input_2], outputs=out)
    
    return model

In [70]:
model = model(vocab_size, embedding_dim, max_len_of_caption)

In [71]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 36, 200)      388200      input_5[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 2048)         0           input_4[0][0]                    
____________________________________________________________________________________________

In [72]:
model.layers[2]
model.layers[2].set_weights([X])
model.layers[2].trainable = False

In [73]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [74]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[np.array(X1), np.array(X2)], np.array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [75]:
epochs = 5
number_pics_per_batch = 3
steps = len(id_caption_dict)//number_pics_per_batch

In [76]:
for i in range(epochs):
    generator = data_generator(id_caption_dict, id_imgarr_dict, word_to_index, max_len_of_caption, number_pics_per_batch)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_' + str(i) + '.h5')

Epoch 1/1
2697/2697 [==============================] - 646s 239ms/step - loss: 3.3852
Epoch 1/1
2697/2697 [==============================] - 652s 242ms/step - loss: 2.7825
Epoch 1/1
2697/2697 [==============================] - 652s 242ms/step - loss: 2.6165
Epoch 1/1
2697/2697 [==============================] - 658s 244ms/step - loss: 2.5207
Epoch 1/1
2697/2697 [==============================] - 655s 243ms/step - loss: 2.4525


In [98]:
def caption(photo,max_length_of_caption):
    in_text = 'start_caption '
    for i in range(max_length_of_caption):
        sequence = [word_to_index[w] for w in in_text.split() if w in word_to_index]
        sequence = pad_sequences([sequence], maxlen=max_length_of_caption)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = index_to_word[yhat]
        in_text += ' ' + word
        if word == 'end_caption':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [78]:
import matplotlib.pyplot as plt

In [99]:
# testing on a pic
pic = '54501196_a9ac9d66f2.jpg'
my_id_imgarr_dict = {}
from keras.preprocessing import image
img = image.load_img(pic,target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predictions = model_inception.predict(x)
x = np.reshape(predictions, predictions.shape[1])
my_id_imgarr_dict[pic]=x
image = x.reshape((1,2048))
print("Caption for image :",caption(image,max_len_of_caption))

Caption for image : a man is walking on a rocky beach .
